In [28]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import re
import pymongo
import time

In [29]:
service = Service("./chromedriver/chromedriver.exe")
options = Options()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")     
driver = webdriver.Chrome(service=service, options=options)    

In [30]:
def finding_raw_urls_from_base(url):       
        driver.get(url)
        WebDriverWait(driver, 60).until(
            EC.presence_of_all_elements_located((By.TAG_NAME, 'a'))
        )        
        html = driver.page_source
        driver.quit()
        
        soup = BeautifulSoup(html, 'html.parser')
        links = soup.find_all('a', href=True)
        raw_urls = []
        for link in links:
            if link['href'].startswith("https://www.moneycontrol.com/news/business/"):
                raw_urls.append(link['href'])
        return list(set(raw_urls))

In [31]:
stock_urls =[]
market_urls = []
next_page_urls = []
other_urls = []
ipo_urls = []
def extract_urls(raw_urls):
        for link in raw_urls:
            if link.startswith("https://www.moneycontrol.com/news/business/stocks/page"):
                next_page_urls.append(link)
            elif link.startswith("https://www.moneycontrol.com/news/business/stocks/"):
                stock_urls.append(link)
            elif link.startswith("https://www.moneycontrol.com/news/business/markets/"):
                market_urls.append(link)
            elif link.startswith("https://www.moneycontrol.com/news/business/ipo/"):
                ipo_urls.append(link)
            else:
                other_urls.append(link)

In [32]:
def extract_data(url):  
        driver.get(url)        
        WebDriverWait(driver, 10).until(
            lambda d: d.execute_script("return document.readyState") == "complete"
        )        
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        news = {}
        
        title = soup.find('h1', class_ = "article_title")
        news.update({"title":f"{title.text.strip()}"})
        
        desc = soup.find('h2', class_ = 'article_desc')
        news.update({"desc":f"{desc.text.strip()}"})
        
        date_time_div = soup.find('div', class_="article_schedule")
        if date_time_div:
            span_tag = date_time_div.find('span')
            date = span_tag.text.strip()
            news.update({"date":f"{date}"})
        
        datetime = date_time_div.text.strip()
        news.update({"datetime":f"{datetime}"})

        return news

In [33]:
for page_counter in range(2,31,1):
    
    raw_urls = finding_raw_urls_from_base(f"https://www.moneycontrol.com/news/business/stocks/page-{page_counter}")
    extract_urls(raw_urls)
    
    regex = r'https:\/\/www\.moneycontrol\.com\/news\/business\/stocks\/[^\/\s]+(?:\.[a-z]{2,6})(?:[\/\?].*)?'
    final_stocks_urls = [url for url in stock_urls if re.match(regex, url)]
    
    batch_data = []
    for url in final_stocks_urls:
        data = extract_data(url)
        batch_data.append(data)
    
    if batch_data:
        myclient = pymongo.MongoClient("mongodb://localhost:27017/", serverSelectionTimeoutMS=5000, socketTimeoutMS=30000)
        mydb = myclient["mydatabase"]
        mycol = mydb['raw_news']

        x = mycol.insert_many(batch_data)
        print(len(x.inserted_ids))
        myclient.close()
    
    batch_data.clear()
    stock_urls.clear()
    time.sleep(2)     

ReadTimeoutError: HTTPConnectionPool(host='localhost', port=58236): Read timed out. (read timeout=120)